In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from functools import reduce
from api_keys import geoapify_key
import json
import requests
import warnings
warnings.filterwarnings('ignore')

In [4]:
dataset_2016 = 'Resources/Burlington_StatsCanada_2016_Census.csv'
dataset_2021 = 'Resources/Burlington_StatsCanada_2021_Census.csv'
private_schools = 'Resources/Private School Boards Ontario.csv'
public_schools = 'Resources/Public School Boards Ontario.csv'
house_sale_median = 'Resources/Burlington_Median_Sale_Price.csv'
house_rent_median = 'Resources/Burlington_Median_Rental_Price.csv'
house_sale_average = 'Resources/Burlington_Avg_Sale_Price.csv'
house_rent_average = 'Resources/Burlington_Avg_Rental_Price.csv'
new_builds_2016 = 'Resources/NewBuillds_Burlington_2016.csv'
new_builds_2021 = 'Resources/New_Builds_Burlington_2021.csv'
property_tax_2016 = 'Resources/2016-Final-Tax-Rates.csv'
property_tax_2021 = 'Resources/2021_Final_Tax_Rates.csv'
df_2016 = pd.read_csv(dataset_2016, encoding = "ISO-8859-1", on_bad_lines = 'skip', low_memory = True, engine = 'python')
df_2021 = pd.read_csv(dataset_2021, encoding = "ISO-8859-1", on_bad_lines = 'skip', low_memory = True, engine = 'python')
df_private_schools = pd.read_csv(private_schools, encoding = "ISO-8859-1", on_bad_lines = 'skip', low_memory = True, engine = 'python')
df_public_schools = pd.read_csv(public_schools, encoding = "ISO-8859-1", on_bad_lines = 'skip', low_memory = True, engine = 'python')
df_sale_median = pd.read_csv(house_sale_median, encoding = "ISO-8859-1", on_bad_lines = 'skip', low_memory = True, engine = 'python')
df_rent_median = pd.read_csv(house_rent_median, encoding = "ISO-8859-1", on_bad_lines = 'skip', low_memory = True, engine = 'python')
df_sale_average = pd.read_csv(house_sale_average, encoding = "ISO-8859-1", on_bad_lines = 'skip', low_memory = True, engine = 'python')
df_rent_average = pd.read_csv(house_rent_average, encoding = "ISO-8859-1", on_bad_lines = 'skip', low_memory = True, engine = 'python')
df_new_builds_2021 = pd.read_csv(new_builds_2021, encoding = "ISO-8859-1", on_bad_lines = 'skip', low_memory = True, engine = 'python')
df_new_builds_2016 = pd.read_csv(new_builds_2016, encoding = "ISO-8859-1", on_bad_lines = 'skip', low_memory = True, engine = 'python')
df_2016_tax = pd.read_csv(property_tax_2016, encoding = "ISO-8859-1", on_bad_lines = 'skip', low_memory = True, engine = 'python')
df_2021_tax = pd.read_csv(property_tax_2021, encoding = "ISO-8859-1", on_bad_lines = 'skip', low_memory = True, engine = 'python')

In [5]:
housing_dfs = [df_sale_median, df_rent_median, df_sale_average, df_rent_average]
df_housing_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Year'], how='outer'), housing_dfs)
years_to_keep = [2016, 2021]
df_housing_merged_filtered = df_housing_merged[df_housing_merged['Year'].isin(years_to_keep)]
renamed_columns = {
    'Sale Price, Median': 'Median Sale Price ($)',
    'Original Price, Median': 'Median Rent ($)',
    'Sale Price, Average': 'Average Sale Price ($)',
    'Original Price, Average': 'Average Rent ($)'
}
df_housing_merged_filtered.rename(columns=renamed_columns, inplace=True)

In [6]:
private_school_count = len(df_private_schools.loc[df_private_schools['City'] == 'Burlington'])


In [7]:
public_school_count = len(df_public_schools.loc[(df_public_schools['City'] == 'Burlington') & (df_public_schools['School Type'] == 'Public')])
catholic_school_count = len(df_public_schools.loc[(df_public_schools['City'] == 'Burlington') & (df_public_schools['School Type'] == 'Catholic')])


In [8]:
new_builds_2016_count = df_new_builds_2016['All'].dropna().sum()
new_builds_2021_count = df_new_builds_2021['All'].dropna().sum()


In [9]:
property_tax_rate_2021 = df_2021_tax['Total Urban Area'][0]
property_tax_rate_2016 = df_2016_tax['Total Urban Area'][0]


In [10]:
target_city = 'Burlington, ON, Canada'

qp = {
    "text": target_city,
    "apiKey": geoapify_key
}
base_url = "https://api.geoapify.com/v2/places"
geocode_url = 'https://api.geoapify.com/v1/geocode/search'
target_url = f"https://api.geoapify.com/v1/geocode/search?text={target_city}&format=json&apiKey={geoapify_key}"
response = requests.get(geocode_url, params=qp).json()
from pprint import pprint




In [11]:
lat = response['features'][0]['properties']['lat']
lon = response['features'][0]['properties']['lon']



In [12]:
#for index, row in leisure_df.iterrows():

#print(leisure_df)
categories = 'leisure'
radius = 183000
limit=300
params = {
    "categories": categories,
    "apiKey": geoapify_key,
    "limit": limit,
    "radius": radius
}
params["filter"] = f'circle:{lon},{lat},{radius}'
params["bias"] = f'proximity:{lon},{lat}'
leisure_dict = requests.get(base_url, params=params).json()

leisure_types = []
for feature in leisure_dict['features']:
    leisure_info = feature['properties']['datasource']['raw']['osm_id']
    if 'city' in feature['properties']:
        leisure_cities = feature['properties']['city']
        if leisure_cities == 'Burlington': 
            leisure_types.append(leisure_info)
    
leisure_count = len(leisure_types)


    

In [13]:
pop_2016 = df_2016.loc[df_2016['Characteristics'] == 'Population; 2016']['Total'].values
income_2016 = df_2016.loc[df_2016['Characteristics'] == '  Average total income of households in 2015 ($)']['Total'].values
house_value_2016 = df_2016.loc[df_2016['Characteristics'] == '  Average value of dwellings ($)']['Total'].values
unemployment_2016 = df_2016.loc[df_2016['Characteristics'] == 'Unemployment rate']['Total'].values
rented_2016 = df_2016.loc[df_2016['Characteristics'] == '  Renter']['Total'].values
owned_2016 = df_2016.loc[df_2016['Characteristics'] == '  Owner']['Total'].values
burlington_2016 = {
    'Year': '2016',
    'City': 'Burlington',
    'Population': pop_2016,
    'Household Income ($)': income_2016,
    'Estimated House Value ($)': house_value_2016,
    'Unemployment Rate (%)': unemployment_2016,
    'Rented Houses': rented_2016,
    'Owned Houses': owned_2016,
    'New Built': new_builds_2016_count,
    'Property Tax Rate (%)': property_tax_rate_2016 * 100
}
burlington_2016_df = pd.DataFrame(burlington_2016, index = [0])


pop_2021 = df_2021.loc[df_2021['Characteristic'] == 'Population, 2021']['Total'].values
income_2021 = df_2021.loc[df_2021['Characteristic'] == '  Average total income of household in 2020 ($)']['Total'].values
house_value_2021 = df_2021.loc[df_2021['Characteristic'] == '  Average value of dwellings ($)']['Total'].values
unemployment_2021 = df_2021.loc[df_2021['Characteristic'] == 'Unemployment rate']['Total'].values
rented_2021 = df_2021.loc[df_2021['Characteristic'] == '  Renter']['Total'].values
owned_2021 = df_2021.loc[df_2021['Characteristic'] == '  Owner']['Total'].values



burlington_2021 = {
    'Year': '2021',
    'City': 'Burlington',
    'Population': pop_2021,
    'Household Income ($)': income_2021,
    'Estimated House Value ($)': house_value_2021,
    'Unemployment Rate (%)': unemployment_2021,
    'Rented Houses': rented_2021,
    'Owned Houses': owned_2021,
    'New Built': new_builds_2021_count,
    'Property Tax Rate (%)': property_tax_rate_2021 * 100
}
burlington_2021_df = pd.DataFrame(burlington_2021, index = [0])


burlington_df_stats = pd.concat([burlington_2016_df, burlington_2021_df], ignore_index=True)
burlington_df_stats.set_index(['Year'])
burlington_df_stats['Year'] = burlington_df_stats['Year'].astype(int)
df_housing_merged_filtered['Year'] = df_housing_merged_filtered['Year'].astype(int)

burlington_df = pd.merge(burlington_df_stats, df_housing_merged_filtered, on='Year')



columns_order = [
    'City', 'Year', 'Population', 'Household Income ($)', 'Average Rent ($)', 'Median Rent ($)',
   'Estimated House Value ($)', 'Unemployment Rate (%)', 'Property Tax Rate (%)',
   'Rented Houses', 'Owned Houses', 'New Built', 'Average Sale Price ($)', 'Median Sale Price ($)'
]

burlington_final_df = burlington_df[columns_order]

burlington_final_df.set_index(['City', 'Year'], inplace = True)

burlington_final_df


Population  Household Income ($)  Average Rent ($)  \
City       Year                                                       
Burlington 2016    183314.0              123402.0              2710   
           2021    186948.0              140200.0              2862   

                 Median Rent ($)  Estimated House Value ($)  \
City       Year                                               
Burlington 2016             2400                   632556.0   
           2021             2800                  1024000.0   

                 Unemployment Rate (%)  Property Tax Rate (%)  Rented Houses  \
City       Year                                                                
Burlington 2016                    5.6               0.871013        16835.0   
           2021                   10.7               0.779583        18220.0   

                 Owned Houses  New Built  Average Sale Price ($)  \
City       Year                                                    
Burlington 2016       54540.0      542.0                  663675   
           2021       54955.0     1494.0                 1081961   

                 Median Sale Price ($)  
City       Year                         
Burlington 2016                 609000  
           2021                 975500

In [14]:
amenities = {
    'City': 'Burlington',
    '# of Public Schools': public_school_count,
    '# of Catholic Schools': catholic_school_count,
    '# of Private Schools': private_school_count,
    '# of Leisure Spaces': leisure_count
}

burlington_amenities_df = pd.DataFrame(amenities, index = [0])
burlington_amenities_df.set_index('City', inplace=True)
burlington_amenities_df

,# of Public Schools,# of Catholic Schools,# of Private Schools,# of Leisure Spaces
City,,,,
Burlington,38,17,2,282
